<a href="https://colab.research.google.com/github/m-fila/uczenie-maszynowe-2021-22/blob/main/06_Bayes_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SPAM vs. naiwny klasyfikator Bayesa**
Autor: Anna Dawid

## Wprowadzenie
Nigeryjski książę wciąż zarabia na użytkownikach elektronicznych skrzynek pocztowych ponad 700 tys. dolarów rocznie ([źródło](https://www.cnbc.com/2019/04/18/nigerian-prince-scams-still-rake-in-over-700000-dollars-a-year.html))! Jak to możliwe?

Pierwsza przyczyna jest natury psychologicznej. Ofiary są poddawane "perfekcyjnej burzy pokuszeń", jak ujął to psycholog w wywiadzie, do którego linka dałam Wam powyżej. Spammerzy łączą granie na ludzkiej chciwości, ale także na pragnieniu bycia bohaterem. W końcu kto nie chciałby zarobić na byciu wspaniałomyślnym i szczodrym? W tej kwestii możemy pracować wyłącznie nad sobą.

Możemy za to pracować nad filtrami antyspamowymi. Użyjemy techniki, która nazywa się "worek ze słowami" (bag of words) w połączeniu z naiwnym klasyfikatorem Bayesa. Choć to prosty klasyfikator, z powodzeniem jest używany współcześnie (np. [SpamAssassin](https://cwiki.apache.org/confluence/display/spamassassin/BayesInSpamAssassin)).

Notebook oparty na tutorialach:
*   https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73
*   https://towardsdatascience.com/spam-filtering-using-naive-bayes-98a341224038


## Import danych treningowych
https://www.kaggle.com/uciml/sms-spam-collection-dataset

To dane przygotowane przez Almeida et al. na podstawie forum brytyjskiego, gdzie użytkownicy skarżą się na spamowe SMSy. Każdy wiersz składa się z kolumny opisującej czy wiadomość jest spamem, czy nie ('spam' czy 'ham'), a druga zawiera treść wiadomości.

Jak na ćwiczeniach o regresji logistycznej klonujemy repozytorium, żeb móc korzystać z zestawu danych:

In [ ]:
# odkomentuj zeby pobrac repozytorium, mozesz tez wgrac samemu odpowiedni plik z danymi
#!git clone https://github.com/m-fila/uczenie-maszynowe-2021-22

Do pracy z danymi tekstowymi bardzo przydatna jest biblioteka [pandas](https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/).

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
mails = pd.read_csv('uczenie-maszynowe-2021-22/dane/spam_dataset.csv', encoding='latin-1')
mails.head()

Wyczyśćmy ten zbiór danych. Usuńmy niepotrzebne kolumny i zastąpmy nazwy 'v1' i 'v2' czymś bardziej przyjaznym.

In [ ]:
mails = mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
mails.head()

In [ ]:
mails = mails.rename(columns={"v1": "klasa", "v2": "tekst"})
mails.head()

Zobaczmy jak wyglądają przykładowe dane o numerze jakimkolwiek.

In [ ]:
id = 57

Treść wiadomości:

In [ ]:
mails['tekst'][id]

Kategoria:

In [ ]:
mails['klasa'][id]

Ile mamy tych maili?

In [ ]:
mails.shape

## Analiza częstości występowania słów w obu klasach za pomocą biblioteki WordCloud

To biblioteka pozwalająca generować śliczne obrazki, na których wielkość słów odpowiada częstości jego występowania w danym zbiorze.

In [ ]:
!pip3 install wordcloud
import wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# najpierw słowa ze spamu
spam_words = " ".join(list(mails [mails['klasa']=='spam']['tekst'] ))
spam_plot = WordCloud(width = 512, height = 512).generate(spam_words)

plt.figure(figsize=(10,8))
plt.imshow(spam_plot)

# teraz słowa z normalnych wiadomosci
ham_words = " ".join(list(mails [mails['klasa']=='ham']['tekst'] ))
ham_plot = WordCloud(width = 512, height = 512).generate(ham_words)

plt.figure(figsize=(10,8))
plt.imshow(ham_plot)

Przygotujmy dane do treningu i testu klasyfikatora:

In [ ]:
from sklearn.model_selection import train_test_split

X = mails.tekst
y = mails.klasa
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

Przekodowujemy wiadomości na wektory cech.  Korzystamy z funkcji: [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# stwórz instancje obiektu CountVectorizer
vectorizer = ...
# naucz vectorizer słownika i przetransformuj dane uczące (jest odpowiednia metoda, która robi dwie rzeczy na raz)
X_train = ...

Wypisz rozmiary danych treningowych:

In [ ]:
print("Dane treningowe: n_samples: %d, n_features: %d" % X_train.shape)

Dane uczące są przechowywane w macierzy rzadkiej (sparse matrix). Proszę podejrzeć jak wyglądają tak przekodowane dane:

In [ ]:
print(X_train[57])

 Wektoryzujemy też dane testowe, wykorzystując już stworzony na podstawie danych treningowych wektor słów:

In [ ]:
# użyj vectorizer żeby przetransformować dane ze zbioru testowego (teraz bez fitowania!)
X_test = ...
print("Dane testowe: n_samples: %d, n_features: %d" % X_test.shape)

Odwrotne mapowanie z cech na słowa:

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names = np.array(feature_names)

Tworzymy instancję i uczymy klasyfikator MultinomialNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# stwórz obiekt klasyfikatora
clf = ...
# naucz klasyfikator na zbiorze uczącym
...

## Ocena jakości: jak zwykle będziemy korzystać z funkcji zaimplementowanych w [sci-kit](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)

In [ ]:
y_pred = ... # obliczamy predykcję dla tekstów ze zbioru testowego

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accur = ... # policz dokładność
print("Dokładność: %0.3f" % accur)
print("Classification report:") 
# wypisz raport klasyfikacji 
print(...)

print("Macierz błędów") 
# wypisz macierz (confusion matrix)
print(...)

Sprawdźmy, czego właściwie maszyna się nauczyła:

In [ ]:
print("Słowa, które z największą pewnością wskazują maszynie, że wiadomość to spam:")
# np.argsort zwraca indeksy w oryginalnej tablicy, które odpowiadają posortowanej tablicy, np.:
# np.argsort([3,1,2]) ---> [1,2,0]
# .coef_[0] zwraca nam tablicę współczynników -- po jednym dla każdej cechy. 
# Wkład do decyzji klasyfikatora jest proporcjonalny do wartości tych współczynników -- większy współczynnik = ważniejsza cecha
top10 = np.argsort(clf.coef_[0])[-10:]
bottom10 = np.argsort(clf.coef_[0])[:10]
print(feature_names[top10])

print("Słowa najmniej istotne przy klasyfikacji:")
print(feature_names[bottom10])

## Zastanówmy się, czy możemy jakoś ułatwić zadanie maszynie, wykorzystując naszą znajomość języka

Poszukajcie słowa, które nie występuje w zbiorze treningowym.

In [ ]:
id = np.where(feature_names == 'supercalifragilisticexpialidocious') #"call mr barosa now to get for free our product"
print(id)

message = ['Life is supercalifragilisticexpialidocious']
our_message = vectorizer.transform(message)
clf.predict(our_message)

### Stemming (nawet nie będę próbować tego tłumaczyć na polski, to [bogate](https://pl.bab.la/slownik/angielski-polski/stemming) znaczeniowo słowo)

Polega na ujednoliceniu słów o tym samym rdzeniu znaczeniowym (o czym maszyna, oczywiście, nie ma szans wiedzieć). Np. dzięki stemmingowi słowa "go", "going" i "goes" są przyporządkowane tylko jednemu słowu "go". Można np. użyć gotowego algorytmu stemmingowego o nazwie [Porter Stemmer](https://tartarus.org/martin/PorterStemmer/).

In [ ]:
id1 = np.where(feature_names == 'going')
id2 = np.where(feature_names == 'go')
print(id1, id2)

In [ ]:
!pip3 install nltk
import nltk
import ssl
# chcemy pobrac wytrenowany dla języka angielskeigo tokenizer Punkt
# poniższe linijki mają pomóc w przypadku problemów z ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
# pobieramy tokenizer
nltk.download('punkt')
# ładujemy przydatne funkcje
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [ ]:
# przykładowy tekst
message = 'Applying classical methods of machine learning to the study of quantum systems (sometimes called quantum machine learning) is the focus of an emergent area of physics research'
# tokenizujemy czyli dzielimy tekst na słowa
words = word_tokenize(message)
print(words)
print()
# stwórz obiekt typu PorterStemmer
stemmer = ...
# użyj nowo stworzonego obiektu, żeby 'dostać temat' każdego ze słów
words = ...
print(words)

Powtórzmy trening i testowanie naszego klasyfikatora na danych poddanych stemmingowi:

### Wracamy do pracy nad zbiorem mejli

In [ ]:
# wczytaj cechy i klasy do zmiennych X i Y a następnie podziel na zbiory uczacy 70% i testowy 30%
X = ...
y = ...
X_train, X_test, y_train, y_test = ...
# stwórz instancje obiektu CountVectorizer
vectorizer = ...
# naucz vectorizer słownika i przetransformuj dane uczące
X_train = ... 
# przetransformuj dane testowe
X_test = ...
# stwórz naiwny bayesowski wielomianowy klasyfikator  
clf = ...
# naucz klasyfiaktor na zbiorze uczącym
...

In [ ]:
# policz predykcję naiwnego Bayesa na zbiorze tesotwym
y_pred = ...
# użyj predykcji aby policzyć dokładność (accuracy)
accur = ... # dokładność
print("Dokładność: %0.3f" % accur)
# wypisz raport klasyfikacji 
print("Classification report:") 
print(...)
 # wypisz macierz (confusion matrix)
print("Macierz błędów")
print(...)

In [ ]:
# Teraz wypiszmy słowa, które były najbardziej i najmniej pomocne przy klasyfikacji.
# Korzystając z wcześniejszych komórek uzupełnij poniższy kod.

# użyj vectorizer żeby dostać nazwy cech i przekonwertuj je do tablicy numpy
...
# print(feature_names[:10])

# wybierz indeksy dla top10 najważniejszych cech
top10 = ...
# wybierz indeksy dla 10 najmniej ważnych cech
bottom10 = ...

print("Słowa, które z największą pewnością wskazują maszynie, że wiadomość to spam:")
# wypisz wyrazy odpowiadające najważniejszym cechom
print(...)
print("Słowa najmniej istotne przy klasyfikacji:")
# wypisz wyrazy odpowiadające najmniej ważnym cechom
print(...)

## Gdybyście byli spammerami... Co moglibyście zrobić, znając tę technikę antyspamową?

### Stosowanie znaków specjalnych zamiast liter

In [ ]:
our_message = vectorizer.transform(['call for free'])
print(clf.predict(our_message))
# podmieniamy literkę a na małpę @
our_tricky_message = vectorizer.transform(['c@ll for free'])
print(clf.predict(our_tricky_message))

### Wysyłanie obrazków z tekstem!
-> nakładki OCR (ang. optical character recognition)

Jakieś inne pomysły? :)

In [ ]:
our_tricky_message = vectorizer.transform(['Call for free sex otherwise you miss a very important meeting'])
print(clf.predict(our_tricky_message))

#### Dlaczego powyższa wiadomość została sklasyfikowana jako pożądana, chociaż jest ewidentnym przykładem spamu?